In [ ]:
news['noun'] = 0
news['propn'] = 0
news['verb'] = 0
news['adj'] = 0
news['adv'] = 0
news['num'] = 0

In [ ]:
def count_pos(row):
    doc = nlp(row.statement)
    for token in doc:
        try:
            row[token.pos_.lower()] += 1
        except:
            pass
        
    return row

In [ ]:
news = news.apply(count_pos,axis=1)

In [ ]:
doc = nlp(news.statement[1])

In [ ]:
news.statement[1]

In [ ]:
for token in doc:
    print (token)
#     for child in token.children:
#         print (child)
    print (get_depth(token))
#     print (token.pos_)
#     break

In [ ]:
subjects_count = {}
for subj in unique_subjects:
    subjects_count[subj] = subjects_list.count(subj)
subjects_count = {value[0]:value[1] for value in sorted(subjects_count.items(), key=lambda x: -x[1])}

In [ ]:
plt.figure(figsize=(8,12))
sns.barplot(x=list(subjects_count.values())[0:50], y=list(subjects_count.keys())[0:50])
plt.show()

In [ ]:
# ohe = OneHotEncoder(drop=[['education_official'],['Russia']])
ohe = OneHotEncoder()
ohe.fit(news_train[['state']])

In [ ]:
ohe.transform(news_train[['state']])[:,1:]

In [ ]:
ohe.transform(news_train[['state']])

In [ ]:
ohe.get_feature_names()

In [ ]:
ohe.categories_

In [ ]:
test_mlb = MultiLabelBinarizer(sparse_output=False)
test_sub = [set(value.split(',')) for value in news_train['subjects']]
test_mlb.fit(test_sub)
# test_sub

In [ ]:
test_sub2 = [set(value.split(',')) for value in news_valid['subjects']]
test_mlb.transform(test_sub2)

In [ ]:
test_sub3 = [set(value.split(',')) for value in news_test['subjects']]
test_mlb.transform(test_sub3)

In [ ]:
test_mlb.classes_

In [ ]:
test_pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline to dummify party column
            ('party', Pipeline([
                ('selector', ItemSelector(key='party')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify context column
            ('context', Pipeline([
                ('selector', ItemSelector(key='context')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify state column
            ('state', Pipeline([
                ('selector', ItemSelector(key='state')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify subjects column
            ('subjects', Pipeline([
                ('selector', ItemSelector(key='subjects')),
                ('ohe', Dummify(multilabel=True)),
            ])),

        ],
    )),
])    

In [ ]:
test_pipeline.fit(news_train)

In [ ]:
test_pipeline.transform(news_train)

In [ ]:
test_pipeline.transform(news_valid)

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('party').named_steps['ohe'].get_feature_names(['party'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('context').named_steps['ohe'].get_feature_names(['context'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('state').named_steps['ohe'].get_feature_names(['state'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('subjects').named_steps['ohe'].get_feature_names(['subject'])

In [ ]:
dict(transformer_pipeline.named_steps['union'].transformer_list).get('text_stats').named_steps['dict_vect'].get_feature_names()

In [ ]:
# logistic regression with count vectorizer
pipeline_text_logr = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
    ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
])
params_text_logr = {'logr__C':np.logspace(3,4,10), 'logr__penalty': ['l1', 'l2']}
text_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
            news_valid['statement'], news_valid['label'])
params = best_params(text_logr_results)
print ('-'*40)
print ('Validation score:', params['score'])
pipeline_text_logr.set_params(**params['params'])
test_score = train_validate(pipeline_text_logr, news_train['statement'], news_train['label'], 
               news_test['statement'], news_test['label'])
print ('Test score:', test_score)

In [ ]:
# TEST DATA - logistic regression with count vectorizer
pipeline_text_logr = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
    ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
])
params_text_logr = {'logr__C':[5994.8425031894085], 'logr__penalty': ['l1']}
text_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
            news_test['statement'], news_test['label'])
best_params(text_logr_results)

In [ ]:
# TEST DATA - logistic regression with tfidf
pipeline_text_logr = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
    ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
])
params_text_logr = {'logr__C':[4641.588833612777], 'logr__penalty': ['l2']}
text_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
            news_test['statement'], news_test['label'])
best_params(text_logr_results)

In [ ]:
# TEST DATA - logistic regression with count vectorizer and selectkbest
pipeline_text_logr = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
    ('select', SelectKBest(k=20000)),
    ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
])
params_text_logr = {'logr__C':[464.15888336127773], 'logr__penalty': ['l1']}
text_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
            news_test['statement'], news_test['label'])
best_params(text_logr_results)

In [ ]:
pipeline_text_logr_final = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(1,2))),
    ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42, C=0.021544346900318832,
                               penalty='l2'))
])
pipeline_text_logr_final.fit(news_train['statement'], news_train['label'])

In [ ]:
pipeline_text_logr_final.named_steps['logr'].classes_

In [ ]:
text_coef = pd.DataFrame(pipeline_text_logr_final.named_steps['logr'].coef_).T
text_coef.columns = pipeline_text_logr_final.named_steps['logr'].classes_
text_coef['ngram'] = pipeline_text_logr_final.named_steps['vect'].get_feature_names()
text_coef.sort_values('pants-fire', ascending=False)

In [ ]:
text_coef.sort_values('true', ascending=False)

In [ ]:
all_logr_coef_mul = pd.DataFrame(all_features_logr_mul.coef_).T
all_logr_coef_mul.columns = all_features_logr_mul.classes_
all_logr_coef_mul['features'] = features
all_logr_coef_mul.sort_values('pants-fire', ascending=False)


In [ ]:
plt.figure(figsize=(16,8))
for lab in order_of_labels:
    plt.plot(subject_cols, subjects_count.loc[lab, :], label=lab, linewidth=1.5)
plt.legend()
plt.title('Plot of subject counts for each class')
plt.xticks(fontsize=7, rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
for lab in order_of_labels:
    plt.plot(party_cols, party_count.loc[lab, :], label=lab, linewidth=1.5)
plt.legend()
plt.title('Plot of party affiliation counts for each class')
plt.xticks(fontsize=10, rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
for lab in order_of_labels:
    plt.plot(party_cols, party_proportion.loc[lab, :], label=lab, linewidth=1.5)
plt.legend()
plt.title('Plot of party affiliation proportion for each class')
plt.ylabel('Proportion of statements from the party')
plt.xlabel('Party')
plt.xticks(fontsize=10, rotation=90)
plt.show()

In [ ]:
party_proportion2 = party_count.apply(calc_proportion,axis=1)
plt.figure(figsize=(16,8))
for party in party_cols:
    if party in ['party_NA', 'party_democrat', 'party_republican']:
        plt.plot(order_of_labels, party_proportion2[party], label=party, linewidth=1.5)
plt.legend()
plt.title('Plot of party affiliation proportion for each class')
plt.xticks(fontsize=10, rotation=90)
plt.show()

In [ ]:
# logistic regression (multinomial) with count vectorizer
# pipeline_text_logr = Pipeline([
#     ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
#     ('logr', LogisticRegression(multi_class='multinomial', solver='saga', random_state=42))
# ])
# params_text_logr = {'logr__C':np.logspace(-2,3,10), 'logr__penalty': ['l1', 'l2']}
# text_cvt_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
#             news_valid['statement'], news_valid['label'], news_test['statement'], news_test['label'])


In [ ]:
# logistic regression with count vectorizer min_df=2
# pipeline_text_logr = Pipeline([
#     ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3), min_df=2)),
#     ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
# ])
# params_text_logr = {'logr__C':np.logspace(3,4,10), 'logr__penalty': ['l1', 'l2']}
# text_cvt2_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
#             news_valid['statement'], news_valid['label'], news_test['statement'], news_test['label'])


In [ ]:
# logistic regression with count vectorizer and selectkbest
# pipeline_text_logr = Pipeline([
#     ('vect', CountVectorizer(stop_words=stopword_list, ngram_range=(2,3))),
#     ('select', SelectKBest(k=20000)),
#     ('logr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
# ])
# params_text_logr = {'logr__C':np.logspace(2,4,10), 'logr__penalty': ['l1', 'l2']}
# text_cvt_kbest_logr_results = grid_search(pipeline_text_logr, params_text_logr, news_train['statement'], news_train['label'], 
#             news_valid['statement'], news_valid['label'], news_test['statement'], news_test['label'])


In [ ]:
# logistic regression with all features + multinomial loss
# all_features_logr = LogisticRegression(multi_class='multinomial', solver='saga', random_state=42)
# params_all_logr = {'C':np.logspace(-2,-1,10), 'penalty': ['l2']}
# all_logr_results = grid_search(all_features_logr, params_all_logr, X_train, news_train['label'], 
#             X_valid, news_valid['label'], X_test, news_test['label'])


In [ ]:
# multinomial loss
# all_features_logr_mul = LogisticRegression(multi_class='multinomial', solver='saga', random_state=42, C=0.03593813663804628,
#                                           penalty='l2')
# # all_features_logr_mul.set_params(**best_params(all_logr_results)['params'])
# all_features_logr_mul.fit(X_train, news_train['label'])

In [ ]:
# BOW + stats
stats_features_logr = LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42)
stats_features_logr.set_params(**best_params(stats_logr_results)['params'])
stats_features_logr.fit(Xs_train, news_train['grouped_label'])


In [ ]:
features_stats = list(itertools.chain(ngrams, text_stats))

In [ ]:
all_logr_coef_stats = pd.DataFrame(stats_features_logr.coef_).T
all_logr_coef_stats.columns = stats_features_logr.classes_
all_logr_coef_stats['features'] = features_stats
all_logr_coef_stats.sort_values('false', ascending=False)


In [ ]:
all_features_svm_final = LinearSVC(random_state=42)
all_features_svm_final.set_params(**best_params(all_svm_results)['params'])
all_features_svm_final.fit(X_train, news_train['label'])

In [ ]:
all_svm_coef = pd.DataFrame(all_features_svm_final.coef_).T
all_svm_coef.columns = all_features_svm_final.classes_
all_svm_coef['features'] = features
all_svm_coef.sort_values('pants-fire', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=news.label.values, yticklabels=news.label.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()